# EPT Configuration

In [ ]:
import os
import time
import glob
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
from itertools import repeat
from ept_config import *

For large point cloud data handling, the [Entwine](https://entwine.io/en/latest/index.html) subset option provides an option to split the overall entwine point tile (EPT) creation task into multiple subsets.  This exercise uses a set of 64 subsets with options defined within individual json configuration files which have defined input output parameters for the EPT build.  After the build process the subsets are merged with:  `entwine merge -c merge.json` where merge.json defines the subset location.

In [ ]:
%%bash
cat merge.json

Generate json subset files for EPT development.

In [ ]:
!python json_gen.py -h 

In [ ]:
!python json_gen.py -i_f ../LPC_Color -o_j . -o_f ../entwine/NC_P3_2015_Coleridge_SE_rgb

In [ ]:
%%bash
cat make_subset.txt 

In [ ]:
in_file = "make_subset.txt"

In [ ]:
start_time = time.time()

with open(in_file) as f:
    lines = [i.rstrip() for i in f.readlines()] 

pool = ProcessPoolExecutor(max_workers=mp.cpu_count())
_ = list(
    pool.map(
        ept_config,
        lines
    )
)
print(f"\n--- {time.time() - start_time} seconds ---")

EPT Generation
Dataset|Computer|Tile Count|Time (Seconds)|Time (Minutes)
---|---|---|---|---
NC P3 2015|RDzur-Mac (M1 Max)|11,242|97246|1620.8
NC_P3_2015_Coleridge_SE|RDzur-Mac (M1 Max)|30|320.53|5.3

In [ ]:
!entwine merge -c merge.json

The following cell creates a text file of the coordinate system for the EPT point cloud with [gdalsrsinfo](https://gdal.org/programs/gdalsrsinfo.html).

In [ ]:
# %%bash
# gdalsrsinfo -e EPSG:6543 -o wkt_simple --single-line | sed 's/\"/\\\"/g' > epsg_6543.txt
!gdalsrsinfo -e EPSG:6543 -o wkt_simple --single-line | sed 's/\"/\\\"/g' > epsg_6543.txt

Run the following command `npm install http-server -g` at the terminal to install the `http-server` in order to view the entwine point tile file.  This procedure follows the [Entwine Quickstart Guide](https://entwine.io/en/latest/quickstart.html) instructions.  If prompted for an updated npm, it should be ok to install the update:  `npm install -g npm@10.8.3`.  Once installed, the npm install command does not need to be run again.

With EPT there are a couple of options to assess the results locally, such as a localhost HTTP server.  The local server can be initiated at the terminal with the following command to start an http-server to view the local data `http-server entwine -p 8080 --cors`.

Once the server is started, open a browser and try to view the EPT result; paste following path into the browser or click on the link:  https://potree.bhinc.com/data/view.html?r=%22http://localhost:8080/NC_P3_2015_Coleridge_SE_rgb%22

The server can be stopped with a ^C at the terminal to interrupt the local http-server.

QGIS is another convenient option for viewing and working with EPT data through the Data Source Manager - Point Cloud, where the EPT can be loaded as a file using the ept.json within the EPT folder.